<a href="https://colab.research.google.com/github/AshandPeach/NLP/blob/main/nlp_AAPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.8 MB/s 
     |████████████████████████████████| 120 kB 93.4 MB/s 
     |████████████████████████████████| 6.6 MB 7.0 MB/s 


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
def apply_to_dict_values(dict, f):
    for key, value in dict.items():
        dict[key] = f(value)

In [ ]:
class AAPDDataset(Dataset):
    """AAPD dataset."""

    def __init__(self, path):
        self.path = path
        self.data = pd.read_csv(self.path, sep='\t', header=None)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_TYPE)

    def __len__(self):
        return self.data.shape[0]

    @staticmethod
    def target_to_tensor(target):
        return torch.tensor([float(label) for label in target])

    def __getitem__(self, idx):
        data = self.tokenizer(self.data.iloc[idx, 1], return_tensors="pt", max_length=512, padding="max_length", truncation=True)
        apply_to_dict_values(data, lambda x: x.flatten())
        return data, AAPDDataset.target_to_tensor(self.data.iloc[idx, 0])

In [ ]:
BERT_TYPE = 'bert-base-uncased'

In [ ]:
# percentage=0.1

In [ ]:
train_dataset = AAPDDataset('/content/drive/MyDrive/nlp/AAPD/train.tsv')
val_dataset = AAPDDataset('/content/drive/MyDrive/nlp/AAPD/validation.tsv')
test_dataset = AAPDDataset('/content/drive/MyDrive/nlp/AAPD/test.tsv')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# test_dataset[0]

In [ ]:
BATCH_SIZE = 128
N_CLASSES = test_dataset[0][1].shape[0]
N_CLASSES

54

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# #for testing
# train_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch.nn as nn

class MultiLabelBert(nn.Module):
    def __init__(self, bert_model, n_classes):
        super(MultiLabelBert, self).__init__()
        self.bert_model = bert_model
        for param in self.bert_model.parameters():
            param.requires_grad = False
        self.n_bert_features = bert_model.pooler.dense.out_features
        self.n_classes = n_classes
        self.dense = nn.Linear(self.n_bert_features, self.n_classes)

    def forward(self, inputs):
        bert_output = self.bert_model(**inputs)
        return self.dense(bert_output.pooler_output)

In [ ]:
import torch.optim as optim

LEARNING_RATE = 1e-4

In [ ]:
model = MultiLabelBert(bert, N_CLASSES).to(device)

In [ ]:
a = []
b = []
batch, target = next(iter(train_dataloader))
a.extend(target.to('cuda').tolist())
apply_to_dict_values(batch, lambda x: x.to(device))
b.extend((torch.sigmoid(model(batch)) > 0.5).type(torch.DoubleTensor).to('cuda').tolist())
len(b), len(b[0])

(128, 54)

In [ ]:
from sklearn.metrics import hamming_loss

# print( "Classification report: \n", (classification_report(a, b)))
print( "F1 micro averaging:",(hamming_loss(a, b)))

F1 micro averaging: 0.49898726851851855


In [ ]:
# from torch.utils.tensorboard import SummaryWriter

In [ ]:
# writer = SummaryWriter('/content/drive/MyDrive/nlp/logs')

In [ ]:
PATH = '/content/drive/MyDrive/nlp/model_saved/tran.pth'

In [ ]:
import tqdm
from sklearn.metrics import f1_score, hamming_loss

def train_model(
        model,
        train_dataloader,
        val_dataloader,
        optimizer,
        criterion,
        n_epochs,
        start_epoch=1):

    for epoch in range(start_epoch, n_epochs+1):
        model.train()
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(train_dataloader, f"Train epoch#{epoch}", leave=True):

            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets.to(device)
            logits = model(batch)
            all_targets.extend(targets.to('cuda').tolist())
            all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cuda').tolist())
            optimizer.zero_grad()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # writer.add_scalar("Batch train loss", loss.item())
            # print("Batch train loss", loss.item())
            #

        print("Accuracy:", torch.sum(torch.tensor(all_preds) == torch.tensor(all_targets)) / len(all_preds))
        print("ave_Train_loss", total_loss / len(train_dataloader))
        print("Train F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        # print("Train Hamming loss",(hamming_loss(all_targets, all_preds)))
            # writer.close()

        model.eval()

        with torch.no_grad():
            total_loss = 0.
            all_targets = []
            all_preds = []
            for batch, targets in tqdm.tqdm(val_dataloader, f"Val epoch#{epoch}", leave=False):
                apply_to_dict_values(batch, lambda x: x.to(device))
                targets = targets.to(device)
                logits = model(batch)
                all_targets.extend(targets.to('cuda').tolist())
                all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cuda').tolist())
                loss = criterion(logits, targets)
                total_loss += loss.item()
            print("Accuracy:", torch.sum(torch.tensor(all_preds) == torch.tensor(all_targets)) / len(all_preds))           
            print("Validation loss", total_loss / len(val_dataloader))
            print("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
            # print("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))    
    
        state = { 'model': model.state_dict(), 'optimizer':optimizer.state_dict(), 'epoch': epoch }   
        torch.save(state, '/content/drive/MyDrive/nlp/model_saved/tran.pth')
        print('model saved for this epoch')

In [ ]:
optimizer = optim.Adam(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE)

In [ ]:
# #normal training 
# ###deleted: model = MultiLabelBert(bert, N_CLASSES).to(device)

# train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10)

In [ ]:
#resume training
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
start_epoch = checkpoint['epoch']+1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 50, start_epoch=start_epoch)

Train epoch#29: 100%|██████████| 421/421 [18:44<00:00,  2.67s/it]


Accuracy: tensor(51.9684)
ave_Train_loss 0.11755786280450799
Train F1 (micro) 0.3209525524108689


Accuracy: tensor(52.1050)
Validation loss 0.11197429988533258
Validation F1 (micro) 0.3943751997443272
model saved for this epoch


Train epoch#30: 100%|██████████| 421/421 [18:43<00:00,  2.67s/it]


Accuracy: tensor(51.9773)
ave_Train_loss 0.11708548116429118
Train F1 (micro) 0.32553184900752485


...

Accuracy: tensor(52.0498)
ave_Train_loss 0.11020526222689984
Train F1 (micro) 0.3783377343074682


Accuracy: tensor(52.1630)
Validation loss 0.10413866862654686
Validation F1 (micro) 0.4390839694656488
model saved for this epoch


Train epoch#50: 100%|██████████| 421/421 [18:41<00:00,  2.66s/it]


Accuracy: tensor(52.0513)
ave_Train_loss 0.10995531071534916
Train F1 (micro) 0.3801911648294521


Accuracy: tensor(52.1730)
Validation loss 0.10430117417126894
Validation F1 (micro) 0.4478694469628286
model saved for this epoch
